Using spacy to indentify UMLS terms. 

Spacy steps:
  1. Load: en_core_sci_lg
  2. Add to pipe: scispacy_linker
  3. Add to pipe: entity_ruler
     - Built with: filtered_umls_atoms

Training EntityLinker with UMLS 

In [1]:
import os
from scispacy.linking import EntityLinker
import spacy
from dotenv import load_dotenv
import pandas as pd
from tqdm.auto import tqdm


dir_downloads = os.path.abspath(f'{os.getcwd()}/../_downloads')

load_dotenv()
EntityLinker()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info 

In [2]:
nlp = spacy.load('en_core_sci_lg')
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True,
                                        "linker_name": "umls",
                                        "threshold":0.85,
                                        "filter_for_definitions": False,
                                        #"disabling": ["tagger", "parser", "attribute_ruler", "lemmatizer"]
                                        })
ruler = nlp.add_pipe("entity_ruler",before="tok2vec")

In [3]:
def create_umls_entity_ruler(ruler, df_cleaned_umls_terms):
    """A dummy docstring."""
    print("Creating UMLS entity ruler...")
    patterns = []
    for _, row in tqdm(df_cleaned_umls_terms.iterrows(), total=df_cleaned_umls_terms.shape[0]):
        patterns.append(
            {"label": row["CUI"], "pattern": row["STR"], "id":row["CUI"]})
    ruler.add_patterns(patterns)

In [4]:
def process_product_ingredient_groups(df_product_ingredient_groups, nlp, conn):
    """process_product_ingredient_groups."""
    print("Processing docs...")
    for df_product_ingredient_group in tqdm(df_product_ingredient_groups.iterrows(),
                                            total=df_product_ingredient_groups.shape[0]):
        doc = nlp(
            str(df_product_ingredient_group[1]['name'])
        )
        product_id = df_product_ingredient_group[1]['id']
        term_type = "PARENT"
        entry_count = 0
        for entry in doc.ents:
            entry_count += 1
            if entry_count != 1:
                term_type="CHILD"

            if entry._.kb_ents:
                umls_cui = entry._.kb_ents[0][0]
                umls_term = entry.text
                data:ProductUmlsTerms = {
                    'product_id':product_id,
                    'term_type':term_type,
                    'umls_cui':umls_cui,
                    'umls_term':umls_term.upper()
                }
                insert_umls_terms(data, conn)
                insert_product_umls_terms(data, conn)

In [ ]:
#spacy.require_gpu()
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
df_cleaned_umls_terms = pd.read_csv(f"{dir_downloads}/umls-data/filtered_umls_atoms.csv")

create_umls_entity_ruler(ruler, df_cleaned_umls_terms)

In [ ]:
df_cleaned_product_ingredient_groups = pd.read_csv(f"{dir_downloads}/product-ingredient-groups/cleaned_product_ingredient_groups.csv")

conn=postgres_connection()
process_product_ingredient_groups(df_cleaned_product_ingredient_groups, nlp, postgres_connection())
conn.close()